In [1]:
import requests 
import elasticsearch

Let's download the collection of the documents for the task:

In [2]:
def download_documents(link):
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()
    
    documents = []
    
    for course in documents_raw:
        course_name = course['course']
    
        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)

    return documents

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'

docs_base = download_documents(docs_url)

In [11]:
docs_base[10]

{'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.',
 'section': 'General course-related questions',
 'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?',
 'course': 'data-engineering-zoomcamp'}

The next step is to index the downloaded data with Elastic Search:

In [13]:
es_client = elasticsearch.Elasticsearch('http://localhost:9200')

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
                },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
                    }
                }
                    }

index_name = "course_questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [23]:
from tqdm.autonotebook import tqdm


for doc in tqdm(docs_base):
    es_client.index(index=index_name, document=doc)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:29<00:00, 32.52it/s]


Let's try to execute some queries

In [40]:
def elastic_func(query, fields, size=5, filter=None):
    search_query = {
        'size': size,
        'query': {
            'bool': {
                'must': {
                    'multi_match': {
                        'query': query,
                        'fields': fields,
                        'type': 'best_fields'
                    }
                }
            }
        }
    }

    if filter:
        search_query['query']['bool']['filter'] = {
            'term': filter
        }
        
    response = es_client.search(index=index_name, body=search_query)
    return response['hits']['hits']

In [41]:
task_query = 'How do I execute a command in a running docker container?'

In [46]:
search_output = elastic_func(task_query, ['question^4', 'text'])
search_output[0]

{'_index': 'course_questions',
 '_id': 'qcn9b5ABT1pLlRzITEx4',
 '_score': 85.48184,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

In [48]:
search_filt_output = elastic_func(task_query, ['question^4', 'text'], size=3, filter={'course': 'machine-learning-zoomcamp'})
search_filt_output[2]

{'_index': 'course_questions',
 '_id': 'ycn9b5ABT1pLlRzIUExd',
 '_score': 51.133015,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

Now creating a prompt for a LLM with the search results

In [68]:
def build_prompt(query, search_results, prompt_template):
    context= '\n\n'.join([
        f"Q: {result['_source']['question']}\nA: {result['_source']['text']}".strip() for result in search_results])
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [69]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

resulting_prompt = build_prompt(task_query, search_filt_output, prompt_template)

In [71]:
print(resulting_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [72]:
len(resulting_prompt)

1462

Calculating tokens with tiktoken:

In [75]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
encoded_prompt = encoding.encode(resulting_prompt)
len(encoded_prompt)

322

Calculating the average price for OpenAI:

In [77]:
150*1000/1000000*5 + 250*1000/1000000*15

4.5